# Transitioning from VTK to PyVista

VTK is primarily developed in C++ and uses chained setter and getter
commands to access data. Instead, PyVista wraps the VTK data types into
numpy arrays so that users can benefit from its bracket syntax and fancy
indexing. This section demonstrates the difference between the two
approaches in a series of examples.

For example, to hard-code values for a `vtkImageData`{.interpreted-text
role="vtk"} data structure using VTK Python\'s bindings, one would write
the following:


In [ ]:
from math import cos, sin

import vtk

Create values for a 300x300 image dataset

In our example, we want to have values from the function

``` python
127.5 + (1.0 + sin(x/25.0)*cos(y/25.0))
```


In [ ]:
values = vtk.vtkDoubleArray()
values.SetName("values")
values.SetNumberOfComponents(1)
values.SetNumberOfTuples(300 * 300)

for x in range(300):
    for y in range(300):
        values.SetValue(x * 300 + y, 127.5 + (1.0 + sin(x / 25.0) * cos(y / 25.0)))

Create the image structure


In [ ]:
image_data = vtk.vtkImageData()
image_data.SetOrigin(0, 0, 0)
image_data.SetSpacing(1, 1, 1)
image_data.SetDimensions(300, 300, 1)

Assign the values to the image


In [ ]:
image_data.GetPointData().SetScalars(values)

As you can see, there is quite a bit of boilerplate that goes into the
creation of a simple `vtkImageData`{.interpreted-text role="vtk"}
dataset. PyVista provides much more concise syntax that is more
\"Pythonic\". The equivalent code in PyVista is:


In [ ]:
import numpy as np
import pyvista as pv

Use the meshgrid function to create 2D \"grids\" of the x and y values.
This section effectively replaces the vtkDoubleArray.


In [ ]:
xi = np.arange(300)
x, y = np.meshgrid(xi, xi)
values = 127.5 + (1.0 + np.sin(x / 25.0) * np.cos(y / 25.0))

Create the grid. Note how the values must use Fortran ordering.


In [ ]:
grid = pv.ImageData(dimensions=(300, 300, 1))
grid.point_data["values"] = values.flatten(order="F")

Here, PyVista has done several things for us:

1.  PyVista combines the dimensionality of the data (in the shape of the
    `numpy.ndarray`{.interpreted-text role="class"}) with the values of
    the data in one line. VTK uses \"tuples\" to describe the shape of
    the data (where it sits in space) and \"components\" to describe the
    type of data (1 = scalars/scalar fields, 2 = vectors/vector fields,
    n = tensors/tensor fields). Here, shape and values are stored
    concretely in one variable.

2.  `pyvista.ImageData`{.interpreted-text role="class"} wraps
    `vtkImageData`{.interpreted-text role="vtk"}, just with a different
    name; they are both containers of evenly spaced points. Your data
    does not have to be an \"image\" to use it with
    `vtkImageData`{.interpreted-text role="vtk"}; rather, like images,
    values in the dataset are evenly spaced apart like pixels in an
    image.

    Furthermore, since we know the container is for uniformly spaced
    data, pyvista sets the origin and spacing by default to `(0, 0, 0)`
    and `(1, 1, 1)`. This is another great thing about PyVista and
    Python! Rather than having to know everything about the VTK library
    up front, you can get started very easily! Once you get more
    familiar with it and need to do something more complex, you can dive
    deeper. For example, changing the origin and spacing is as simple
    as:

    ``` python
    grid.origin = (10, 20, 10)
    grid.spacing = (2, 3, 5)
    ```

3.  The name for the
    `point_array <pyvista.point_array>`{.interpreted-text role="attr"}
    is given directly in dictionary-style fashion. Also, since VTK
    stores data on the heap (linear segments of RAM; a C++ concept), the
    data must be flattened and put in Fortran ordering (which controls
    how multidimensional data is laid out in physically 1d memory; numpy
    uses \"C\"-style memory layout by default). This is why in our
    earlier example, the first argument to `SetValue()` was written as
    `x*300 + y`. Here, numpy takes care of this for us quite nicely and
    it\'s made more explicit in the code, following the Python best
    practice of \"Explicit is better than implicit\".

Finally, with PyVista, each geometry class contains methods that allow
you to immediately plot the mesh without also setting up the plot. For
example, in VTK you would have to do:

``` python
actor = vtk.vtkImageActor()
actor.GetMapper().SetInputData(image_data)
ren = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren)
renWin.SetWindowName('ReadSTL')
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)
ren.AddActor(actor)
iren.Initialize()
renWin.Render()
iren.Start()
```


However, with PyVista you only need:


In [ ]:
grid.plot(cpos="xy", show_scalar_bar=False, cmap="coolwarm")

# PointSet Construction

PyVista heavily relies on NumPy to efficiently allocate and access
VTK\'s C arrays. For example, to create an array of points within VTK
one would normally loop through all the points of a list and supply that
to a `vtkPoints`{.interpreted-text role="vtk"} class. For example:


In [ ]:
vtk_array = vtk.vtkDoubleArray()
vtk_array.SetNumberOfComponents(3)
vtk_array.SetNumberOfValues(9)
vtk_array.SetValue(0, 0)
vtk_array.SetValue(1, 0)
vtk_array.SetValue(2, 0)
vtk_array.SetValue(3, 1)
vtk_array.SetValue(4, 0)
vtk_array.SetValue(5, 0)
vtk_array.SetValue(6, 0.5)
vtk_array.SetValue(7, 0.667)
vtk_array.SetValue(8, 0)
vtk_points = vtk.vtkPoints()
vtk_points.SetData(vtk_array)

To do the same within PyVista, you simply need to create a NumPy array:


In [ ]:
np_points = np.array([[0, 0, 0], [1, 0, 0], [0.5, 0.667, 0]])

::: note
::: title
Note
:::

You can use `pyvista.vtk_points`{.interpreted-text role="func"} to
construct a `vtkPoints`{.interpreted-text role="vtk"} object, but this
is unnecessary in almost all situations.
:::

Since the end goal is to construct a `pyvista.DataSet
<pyvista.core.dataset.DataSet>`{.interpreted-text role="class"}, you
would simply pass the `np_points` array to the
`pyvista.PolyData`{.interpreted-text role="class"} constructor:


In [ ]:
poly_data = pv.PolyData(np_points)

Whereas in VTK you would have to do:


In [ ]:
vtk_poly_data = vtk.vtkPolyData()
vtk_poly_data.SetPoints(vtk_points)

The same goes with assigning face or cell connectivity/topology. With
VTK you would normally have to loop using
`InsertNextCell`{.interpreted-text role="func"} and
`InsertCellPoint`{.interpreted-text role="func"}. For example, to create
a single cell (triangle) and then assign it to
`vtkPolyData`{.interpreted-text role="vtk"}:


In [ ]:
cell_arr = vtk.vtkCellArray()
cell_arr.InsertNextCell(3)
cell_arr.InsertCellPoint(0)
cell_arr.InsertCellPoint(1)
cell_arr.InsertCellPoint(2)
vtk_poly_data.SetPolys(cell_arr)

In PyVista, we can assign this directly in the constructor and then
access it (or change it) from the `faces
<pyvista.PolyData.faces>`{.interpreted-text role="attr"} attribute.


In [ ]:
faces = np.array([3, 0, 1, 2])
poly_data = pv.PolyData(np_points, faces)
poly_data.faces

# PyVista Tradeoffs

While most features can, not everything can be simplified in PyVista
without losing functionality or performance.

In the `collision <pyvista.PolyDataFilters.collision>`{.interpreted-text
role="class"} filter, we demonstrate how to calculate the collision
between two meshes. For example:


In [ ]:
# create a default sphere and a shifted sphere
mesh_a = pv.Sphere()
mesh_b = pv.Sphere(center=(-0.4, 0, 0))
out, n_coll = mesh_a.collision(mesh_b, generate_scalars=True, contact_mode=2)

In [ ]:
pl = pv.Plotter()
pl.add_mesh(out)
pl.add_mesh(mesh_b, style="wireframe", color="k")
pl.camera_position = "xy"
pl.show()

Under the hood, the collision filter detects mesh collisions using
oriented bounding box (OBB) trees. For a single collision, this filter
is as performant as the VTK counterpart, but when computing multiple
collisions with the same meshes, as in the [Collision
Example](https://docs.pyvista.org/examples/01-filter/collisions.html)
example, it is more efficient to use the
[vtkCollisionDetectionFilter](https://vtk.org/doc/nightly/html/classvtkCollisionDetectionFilter.html),
as the OBB tree is computed once for each mesh. In most cases, pure
PyVista is sufficient for most data science, but there are times when
you may want to use VTK classes directly.


```{=html}
<center>
  <a target="_blank" href="https://colab.research.google.com/github/pyvista/pyvista-tutorial/blob/gh-pages/notebooks/tutorial/06_vtk/a_1_transition_vtk.ipynb">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/ width="150px">
  </a>
</center>
```
